In [93]:
import pandas as pd
import requests


<pre>

Variables(10):

1. Interest Rate ✓
2. Real GDP Growth✓ 
3. Federal Budget Deficit (% GDP) ✓
4. Debt Held by the Public (% GDP) ✓
5. Federal Revenue Growth (YoY) ✓
6. Federal Spending Growth (YoY) ✓ 
7. Unemployment Rate ✓ 
8. Population Age 65+ (%) ✓
9. Inflation Rate (CPI YoY) ✓
10. National Debt ✓


</pre>

**National Debt**

In [107]:
#api key
#endpoint and parameters
endpoint = 'https://api.stlouisfed.org/fred/series/observations'


**DataFrame for Interest Rates**

In [193]:
params_1 = {
    'series_id' : 'FEDFUNDS',
    'api_key' : key,
    'file_type' : 'json',
    'observation_start' : '1999-01-01',
    'observation_end' : '2024-01-01'
}

response_1 = requests.get(endpoint,params_1)
data = response_1.json()
df_1 = pd.DataFrame(data['observations'])
df_1['date'] = pd.to_datetime(df_1['date'])
df_1['value'] = pd.to_numeric(df_1['value'], errors='coerce')  # Handle any "." values
df_1.set_index('date', inplace=True)

# Convert quarterly GDP to annual GDP (use .sum() for total annual GDP)
df_interest = df_1[['value']].resample('YE').sum()  # or .mean() if you prefer average

# Step 1: Initialize an empty list
pct_changes_interest = []

# Step 2: Convert index to year for cleaner labeling (optional)
df_interest.index = df_interest.index.year

# Step 3: Loop through and compute % change
years = df_interest.index.tolist()
values = df_interest['value'].tolist()

for i in range(1, len(values)):
    prev = values[i - 1]
    curr = values[i]
    if prev != 0:
        pct_change_interest = ((curr - prev) / prev) * 100
    else:
        pct_change_intereset = None  # or np.nan to avoid division by zero
    pct_changes_interest.append((years[i], pct_change_interest))  # Skip year 0 because no prior data

# Step 4: Turn into DataFrame
df_pct_change_interest = pd.DataFrame(pct_changes_interest, columns=['Year', 'Interest Rate'])
#df_pct_change_interest

**DataFrame for GDP**

In [189]:
params_2= {
    'series_id': 'GDP',
    'api_key': key,
    'file_type': 'json',
    'observation_start': '1999-01-01',
    'observation_end': '2024-01-01'
}

response = requests.get(endpoint,params_2)
data = response.json()
df_2 = pd.DataFrame(data['observations'])
df_2['date'] = pd.to_datetime(df_2['date'])
df_2['value'] = pd.to_numeric(df_2['value'], errors='coerce')  # Handle any "." values
df_2.set_index('date', inplace=True)

# Convert quarterly GDP to annual GDP (use .sum() for total annual GDP)
df_annual = df_2[['value']].resample('YE').sum()  # or .mean() if you prefer average

# Step 1: Initialize an empty list
pct_changes = []

# Step 2: Convert index to year for cleaner labeling (optional)
df_annual.index = df_annual.index.year

# Step 3: Loop through and compute % change
years = df_annual.index.tolist()
values = df_annual['value'].tolist()

for i in range(1, len(values)):
    prev = values[i - 1]
    curr = values[i]
    if prev != 0:
        pct_change = ((curr - prev) / prev) * 100
    else:
        pct_change = None  # or np.nan to avoid division by zero
    pct_changes.append((years[i], pct_change))  # Skip year 0 because no prior data

# Step 4: Turn into DataFrame
df_pct_change = pd.DataFrame(pct_changes, columns=['Year', 'GDP'])


**Percentage Change of Federal Budget Deficit**
<pre>
if negative, means that government used more money then it had
</pre>

In [187]:
params_3 = {
    'series_id': 'MTSDS133FMS',
    'api_key': key,
    'file_type': 'json',
    'observation_start': '1999-01-01',
    'observation_end': '2024-01-01'
}

response_deficit = requests.get(endpoint,params_3)
data_deficit = response_deficit.json()

df_deficit = pd.DataFrame(data_deficit['observations'])
df_deficit['date'] = pd.to_datetime(df_deficit['date'])
df_deficit['value'] = pd.to_numeric(df_deficit['value'], errors='coerce')
df_deficit.set_index('date', inplace=True)

df_GDP_Deficit = df_deficit[['value']].resample('YE').sum()

# Step 1: Initialize an empty list
pct_changes_gdp = []

# Step 2: Convert index to year for cleaner labeling (optional)
df_GDP_Deficit.index = df_GDP_Deficit.index.year

# Step 3: Loop through and compute % change
years = df_GDP_Deficit.index.tolist()
values = df_GDP_Deficit['value'].tolist()

for i in range(1, len(values)):
    prev = values[i - 1]
    curr = values[i]
    if prev != 0:
        pct_change_gdp = ((curr - prev) / prev) * 100
    else:
        pct_change_gdp = None  # or np.nan to avoid division by zero
    pct_changes_gdp.append((years[i], pct_change_gdp))  # Skip year 0 because no prior data

# Step 4: Turn into DataFrame
df_pct_change_gdp = pd.DataFrame(pct_changes_gdp, columns=['Year', 'Percentage Change of Federal Budget Deficit'])

**Percentage of Debt held by the people**

In [204]:
params_4 = {
    'series_id': 'FYGFDPUN',
    'api_key': key,
    'file_type': 'json',
    'observation_start': '1999-01-01',
    'observation_end': '2024-01-01'
}

response_debtHeldbyPeople = requests.get(endpoint,params_4)
data_debtHeldbyPeople = response_debtHeldbyPeople.json()

df_debtbyPeople = pd.DataFrame(data_debtHeldbyPeople['observations'])
df_debtbyPeople['date'] = pd.to_datetime(df_debtbyPeople['date'])
df_debtbyPeople['value'] = pd.to_numeric(df_debtbyPeople['value'], errors='coerce')
df_debtbyPeople.set_index('date', inplace=True)

df_debtHeldbyPeople = df_debtbyPeople[['value']].resample('YE').last()

# Step 1: Initialize an empty list
debtHeldbyPeople_changes = []

# Step 2: Convert index to year for cleaner labeling (optional)
df_debtHeldbyPeople.index = df_debtHeldbyPeople.index.year

# Step 3: Loop through and compute % change
years = df_debtHeldbyPeople.index.tolist()
values = df_debtHeldbyPeople['value'].tolist()

for i in range(1, len(values)):
    prev = values[i - 1]
    curr = values[i]
    if prev != 0:
        debtHeldbyPeople_change = ((curr - prev) / prev) * 100
    else:
        debtHeldbyPeople_changes = None  # or np.nan to avoid division by zero
    debtHeldbyPeople_changes.append((years[i], debtHeldbyPeople_change))  # Skip year 0 because no prior data

# Step 4: Turn into DataFrame
df_debtHeldbyPeople_changes = pd.DataFrame(debtHeldbyPeople_changes, columns=['Year', 'Percentage of Debt held by the people'])
df_debtHeldbyPeople_changes


**US Federal Revenue**

In [206]:
params_5 = {
    'series_id': 'FGRECPT',
    'api_key': key,
    'file_type': 'json',
    'observation_start': '1999-01-01',
    'observation_end': '2024-01-01'
}

response_revenue= requests.get(endpoint,params_5)
data_revenue = response_revenue.json()

df_revenue = pd.DataFrame(data_revenue['observations'])
df_revenue['date'] = pd.to_datetime(df_revenue['date'])
df_revenue['value'] = pd.to_numeric(df_revenue['value'], errors='coerce')
df_revenue.set_index('date', inplace=True)

df_FederalRevenue = df_revenue[['value']].resample('YE').last()

# Step 1: Initialize an empty list
federalRevenue_changes = []

# Step 2: Convert index to year for cleaner labeling (optional)
df_FederalRevenue.index = df_FederalRevenue.index.year

# Step 3: Loop through and compute % change
years = df_FederalRevenue.index.tolist()
values = df_FederalRevenue['value'].tolist()

for i in range(1, len(values)):
    prev = values[i - 1]
    curr = values[i]
    if prev != 0:
        federalRevenue_change = ((curr - prev) / prev) * 100
    else:
        federalRevenue_changes = None  # or np.nan to avoid division by zero
    federalRevenue_changes.append((years[i], federalRevenue_change))  # Skip year 0 because no prior data

# Step 4: Turn into DataFrame
df_FederalRevenue_changes = pd.DataFrame(federalRevenue_changes, columns=['Year', 'US Federal Revenue'])
#df_FederalRevenue_changes

**Federal Spending Growth**

In [212]:
params_6 = {
    'series_id': 'FYONGDA188S',
    'api_key': key,
    'file_type': 'json',
    'observation_start': '1999-01-01',
    'observation_end': '2024-01-01'
}

response_spending= requests.get(endpoint,params_6)
data_spending = response_spending.json()

df_spending = pd.DataFrame(data_spending['observations'])
df_spending['date'] = pd.to_datetime(df_spending['date'])
df_spending['value'] = pd.to_numeric(df_spending['value'], errors='coerce')
df_spending.set_index('date', inplace=True)

df_FederalSpending = df_spending[['value']].resample('YE').last()


# Step 1: Initialize an empty list
federalSpending_changes = []

# Step 2: Convert index to year for cleaner labeling (optional)
df_FederalSpending.index = df_FederalSpending.index.year

# Step 3: Loop through and compute % change
years = df_FederalSpending.index.tolist()
values = df_FederalSpending['value'].tolist()

for i in range(1, len(values)):
    prev = values[i - 1]
    curr = values[i]
    if prev != 0:
        federalSpending_change = ((curr - prev) / prev) * 100
    else:
        federalSpending_changes = None  # or np.nan to avoid division by zero
    federalSpending_changes.append((years[i], federalSpending_change))  # Skip year 0 because no prior data

# Step 4: Turn into DataFrame
df_FederalSpending_changes = pd.DataFrame(federalSpending_changes, columns=['Year', 'Federal Spending Growth'])
#df_FederalSpending_changes

**Unemployment Rate**

In [220]:
params_7 = {
    'series_id': 'UNRATE',
    'api_key': key,
    'file_type': 'json',
    'observation_start': '1999-01-01',
    'observation_end': '2024-01-01'
}

response_unemployment= requests.get(endpoint,params_7)
data_unemployment = response_unemployment.json()

df_unemployment = pd.DataFrame(data_unemployment['observations'])
df_unemployment['date'] = pd.to_datetime(df_unemployment['date'])
df_unemployment['value'] = pd.to_numeric(df_unemployment['value'], errors='coerce')
df_unemployment.set_index('date', inplace=True)

df_unemployment = df_unemployment[['value']].resample('YE').last()


# Step 1: Initialize an empty list
unemployment_changes = []

# Step 2: Convert index to year for cleaner labeling (optional)
df_unemployment.index = df_unemployment.index.year

# Step 3: Loop through and compute % change
years = df_unemployment.index.tolist()
values = df_unemployment['value'].tolist()

for i in range(1, len(values)):
    prev = values[i - 1]
    curr = values[i]
    if prev != 0:
        unemployment_change = ((curr - prev) / prev) * 100
    else:
        unemployment_changes = None  # or np.nan to avoid division by zero
    unemployment_changes.append((years[i], unemployment_change))  # Skip year 0 because no prior data

# Step 4: Turn into DataFrame
df_unemployment_changes = pd.DataFrame(unemployment_changes, columns=['Year', 'Unemployment Rate'])
#df_unemployment_changes

**Population Age 65+**

In [222]:
params_8 = {
    'series_id': 'SPPOP65UPTOZSUSA',
    'api_key': key,
    'file_type': 'json',
    'observation_start': '1999-01-01',
    'observation_end': '2024-01-01'
}

response_over65= requests.get(endpoint,params_8)
data_over65 = response_over65.json()

df_over65 = pd.DataFrame(data_over65['observations'])
df_over65['date'] = pd.to_datetime(df_over65['date'])
df_over65['value'] = pd.to_numeric(df_over65['value'], errors='coerce')
df_over65.set_index('date', inplace=True)

df_over65 = df_over65[['value']].resample('YE').last()


# Step 1: Initialize an empty list
over65_changes = []

# Step 2: Convert index to year for cleaner labeling (optional)
df_over65.index = df_over65.index.year

# Step 3: Loop through and compute % change
years = df_over65.index.tolist()
values = df_over65['value'].tolist()

for i in range(1, len(values)):
    prev = values[i - 1]
    curr = values[i]
    if prev != 0:
        over65_change = ((curr - prev) / prev) * 100
    else:
        over65_changes = None  # or np.nan to avoid division by zero
    over65_changes.append((years[i], over65_change))  # Skip year 0 because no prior data

# Step 4: Turn into DataFrame
df_over65_changes = pd.DataFrame(over65_changes, columns=['Year', 'Population Age 65+'])
#df_over65_changes

**Inflation Rate**

In [226]:
params_9 = {
    'series_id': 'CPIAUCSL',
    'api_key': key,
    'file_type': 'json',
    'observation_start': '1999-01-01',
    'observation_end': '2024-01-01'
}

response_inflation= requests.get(endpoint,params_9)
data_inflation = response_inflation.json()

df_inflation = pd.DataFrame(data_inflation['observations'])
df_inflation['date'] = pd.to_datetime(df_inflation['date'])
df_inflation['value'] = pd.to_numeric(df_inflation['value'], errors='coerce')
df_inflation.set_index('date', inplace=True)

df_inflation = df_inflation[['value']].resample('YE').last()


# Step 1: Initialize an empty list
inflation_changes = []

# Step 2: Convert index to year for cleaner labeling (optional)
df_inflation.index = df_inflation.index.year

# Step 3: Loop through and compute % change
years = df_inflation.index.tolist()
values = df_inflation['value'].tolist()

for i in range(1, len(values)):
    prev = values[i - 1]
    curr = values[i]
    if prev != 0:
        inflation_change = ((curr - prev) / prev) * 100
    else:
        inflation_changes = None  # or np.nan to avoid division by zero
    inflation_changes.append((years[i], inflation_change))  # Skip year 0 because no prior data

# Step 4: Turn into DataFrame
df_inflation_changes = pd.DataFrame(inflation_changes, columns=['Year', 'Inflation'])
#df_inflation_changes

**National Debt**

In [228]:
params_10 = {
    'series_id': 'GFDEBTN',
    'api_key': key,
    'file_type': 'json',
    'observation_start': '1999-01-01',
    'observation_end': '2024-01-01'
}

response_nationalDebt= requests.get(endpoint,params_10)
data_nationalDebt = response_nationalDebt.json()

df_nationalDebt = pd.DataFrame(data_nationalDebt['observations'])
df_nationalDebt['date'] = pd.to_datetime(df_nationalDebt['date'])
df_nationalDebt['value'] = pd.to_numeric(df_nationalDebt['value'], errors='coerce')
df_nationalDebt.set_index('date', inplace=True)

df_nationalDebt = df_nationalDebt[['value']].resample('YE').last()


# Step 1: Initialize an empty list
nationalDebt_changes = []

# Step 2: Convert index to year for cleaner labeling (optional)
df_nationalDebt.index = df_nationalDebt.index.year

# Step 3: Loop through and compute % change
years = df_nationalDebt.index.tolist()
values = df_nationalDebt['value'].tolist()

for i in range(1, len(values)):
    prev = values[i - 1]
    curr = values[i]
    if prev != 0:
        nationalDebt_change = ((curr - prev) / prev) * 100
    else:
        nationalDebt_changes = None  # or np.nan to avoid division by zero
    nationalDebt_changes.append((years[i], nationalDebt_change))  # Skip year 0 because no prior data

# Step 4: Turn into DataFrame
df_nationalDebt_changes = pd.DataFrame(nationalDebt_changes, columns=['Year', 'National Debt'])
#df_nationalDebt_changes

<pre>
Names of all of the DataFrames:

1. Interest Rate: df_pct_change_interest
2. Real GDP Growth: df_pct_change
3. Federal Budget Deficit (% GDP): df_pct_change_gdp
4. Debt Held by the Public (% GDP): df_debtHeldbyPeople_changes
5. Federal Revenue Growth (YoY): df_FederalRevenue_changes
6. Federal Spending Growth (YoY): df_FederalSpending_changes
7. Unemployment Rate: df_unemployment_changes
8. Population Age 65+: df_over65_changes
9. Inflation Rate: df_inflation_changes
10. National Debt: df_nationalDebt_changes

</pre>

**Merge all DataFrames**

In [262]:
# 1. Put your dfs and their desired names in parallel lists:
dfs = [
    df_pct_change_interest,
    df_pct_change,
    df_pct_change_gdp,
    df_debtHeldbyPeople_changes,
    df_FederalRevenue_changes,
    df_FederalSpending_changes,
    df_unemployment_changes,
    df_over65_changes,
    df_inflation_changes,
    df_nationalDebt_changes
]
names = [
    'Interest Rate',
    'Real GDP Growth',
    'Federal Budget Deficit',
    'Debt Held by the Public',
    'Federal Revenue Growth',
    'Federal Spending Growth',
    'Unemployment Rate',
    'Population Age 65+',
    'Inflation Rate',
    'National Debt'
]

# 2. Seed df_main with the first one, renamed:
df_main = dfs[0].rename(columns={'YoY_Percent_Change': names[0]})

# 3. Loop over the rest, rename, and join:
for df, name in zip(dfs[1:], names[1:]):
    df_renamed = df.rename(columns={'YoY_Percent_Change': name})
    df_main = df_main.join(df_renamed, how='left', rsuffix=f'_{i}')

# Now df_main.columns == ['interest_pct','budget_pct','gdp_pct','debt_per_person_pct', …]

df_main = df_main.drop('Year_25', axis=1)
df_main = df_main.drop(index=25, axis=0)
df_main

,Year,Interest Rate,Real GDP Growth,Federal Budget Deficit,Debt Held by the Public,Federal Revenue Growth,Federal Spending Growth,Unemployment Rate,Population Age 65+,Inflation,National Debt
0,2000,25.469484,6.435149,60.669794,-8.127775,6.774772,-1.237059,-2.500000,-0.449737,3.027249,-1.971489
1,2001,-37.658693,3.228744,-63.012070,-0.579545,-4.026190,0.873733,46.153846,-0.367612,3.027249,4.966660
2,2002,-57.127546,3.280869,-344.710013,7.473338,-6.573178,4.518284,5.263158,-0.228583,3.027249,7.777787
3,2003,-32.350000,4.825108,72.999207,10.875435,3.281637,2.465837,-5.000000,-0.083851,3.027249,9.245771
4,2004,19.660015,6.640332,-0.005262,8.958782,7.836251,-0.455097,-5.263158,0.065173,3.027249,8.547901
5,2005,138.171711,6.728232,-19.361314,6.995696,14.008195,1.015382,-9.259259,0.162903,3.027249,7.560021
6,2006,54.486515,5.954247,-34.980530,3.957933,8.527005,1.370933,-10.204082,0.439149,3.027249,6.239721
7,2007,1.107940,4.767399,-10.170682,4.796369,3.166297,-1.903237,13.636364,1.035028,3.027249,6.324123
8,2008,-61.597211,2.042491,262.067149,24.071139,-6.704631,7.115520,46.000000,1.434211,3.027249,15.934615
9,2009,-91.699092,-1.975608,116.218079,22.623363,-7.907399,20.319163,35.616438,1.477034,3.027249,15.061433


**Find confounders then construct a hypothesized structural causal model**

<pre>
Some Confounders include: 

1. ideology
2. interest rates
3. global shocks

(add more but this is fine for now)